## Neural Style Transfer

In [29]:
# For this script I used some of the 2nd edition code found here:
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter12_part03_neural-style-transfer.ipynb

In [30]:
# calculate image height based on set image width
from tensorflow import keras

base_image_path = keras.utils.get_file('sf.jpg', origin='https://img-datasets.s3.amazonaws.com/sf.jpg')
style_image_path = keras.utils.get_file('starry_night.jpg', origin = 'https://img-datasets.s3.amazonaws.com/starry_night.jpg')

width, height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(width * img_height / height)

In [31]:
# auxillary functions
import numpy as np

def preprocess_image(image_path):
    img = keras.utils.load_img(image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

# reverses processes done in vgg19.preprocess_input()
def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    # zero-centering via removing the mean pizel value from ImageNet
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    # bgr to rgb
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [41]:
# load the pretrained VGG19 network
model = keras.applications.vgg19.VGG19(weights='imagenet', include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [33]:
# define loss functions
import tensorflow as tf

def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    return tf.matmul(features, tf.transpose(features))

def total_variation_loss(x):
    a = tf.square(x[:, :img_height - 1, :img_width - 1, :] -
                x[:, 1:, :img_width - 1, :])
    b = tf.square(x[:, :img_height - 1, :img_width - 1, :] -
                 x[:, :img_height - 1, 1:, :])
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [40]:
# combine loss functions into final loss
content_weight = 2.5e-8
style_weight = 1e-6
total_variation_weight = 1e-6

content_layer_name = 'block5_conv2'
style_layer_names = ['block1_conv1',
                     'block2_conv1',
                     'block3_conv1',
                     'block4_conv1',
                     'block5_conv1']

style_weight /= len(style_layer_names)

def compute_loss(combination, base, style):
    input_tensor = tf.concat([base, style, combination], axis=0)
    features = feature_extractor(input_tensor)
    
    loss = tf.zeros(shape=())

    # add content loss
    layer_features = features[content_layer_name]
    base_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + (content_weight * content_loss(base_features,
                                        combination_features))

    # add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        loss = loss + (style_weight * style_loss(style_features,
                                                combination_features))
        
    # add total variation loss
    loss = loss + (total_variation_weight * total_variation_loss(combination))
    
    return loss

In [ ]:
# create combination image via gradient-descent
import time

@tf.function
def compute_loss_and_grads(combination, base, style):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination, base, style)
    grads = tape.gradient(loss, combination)
    return loss, grads
    
optimizer = keras.optimizers.legacy.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 100.0, decay_steps=100, decay_rate=0.96
    )
)

base = preprocess_image(base_image_path)
style = preprocess_image(style_image_path)
combination = tf.Variable(preprocess_image(base_image_path))

iterations = 200
for i in range(1, iterations + 1):
    start_time = time.time()
    
    loss, grads = compute_loss_and_grads(combination, base, style)
    optimizer.apply_gradients([(grads, combination)])
    
    if i % 10 == 0 or i == 1:
        img = deprocess_image(combination.numpy())
        fname = f'combination_image_at_iteration_{i}.png'
        keras.utils.save_img(fname, img)
        
    print(f'Iteration {i}: loss={loss:.2f}, time={(time.time() - start_time):.1f}s')